In [1]:
%defaultDatasource jdbc:h2:mem:db

Criação das tabelas seguindo o UML

In [2]:
CREATE TABLE Paciente (
  Id VARCHAR(16) NOT NULL,
  Diagnostico VARCHAR(1) NOT NULL,
  Raio DECIMAL(20,10) NOT NULL,
  Textura DECIMAL(20,10) NOT NULL,
  Perimetro DECIMAL(20,10) NOT NULL,
  Area DECIMAL(20,10) NOT NULL,
  Suavidade DECIMAL(20,10) NOT NULL,
  Compacidade DECIMAL(20,10) NOT NULL,
  Concavidade DECIMAL(20,10) NOT NULL,
  PontosConcavos DECIMAL(20,10) NOT NULL,
  Simetria DECIMAL(20,10) NOT NULL,
  DimensaoFractal DECIMAL(20,10) NOT NULL,  
  PRIMARY KEY(Id)
) AS SELECT
    id,
    diagnosis,
    radius_mean,
    texture_mean,
    perimeter_mean,
    area_mean,
    smoothness_mean,
    compactness_mean,
    concavity_mean,
    concave_points_mean,
    symmetry_mean,
    fractal_dimension_mean
FROM CSVREAD('data.csv');

CREATE TABLE Erro (
  IdPaciente VARCHAR(16) NOT NULL,
  Raio_erro DECIMAL(20,10) NOT NULL,
  Textura_erro DECIMAL(20,10) NOT NULL,
  Perimetro_erro DECIMAL(20,10) NOT NULL,
  Area_erro DECIMAL(20,10) NOT NULL,
  Suavidade_erro DECIMAL(20,10) NOT NULL,
  Compacidade_erro DECIMAL(20,10) NOT NULL,
  Concavidade_erro DECIMAL(20,10) NOT NULL,
  PontosConcavos_erro DECIMAL(20,10) NOT NULL,
  Simetria_erro DECIMAL(20,10) NOT NULL,
  DimensaoFractal_erro DECIMAL(20,10) NOT NULL,
  PRIMARY KEY(IdPaciente),
  FOREIGN KEY(IdPaciente)
    REFERENCES Paciente(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    id,
    radius_se,
    texture_se,
    perimeter_se,
    area_se,
    smoothness_se,
    compactness_se,
    concavity_se,
    concave_points_se,
    symmetry_se,
    fractal_dimension_se
FROM CSVREAD('data.csv');

In [3]:
SELECT * FROM Paciente;
SELECT * FROM Erro;

Consulta simples para descobrirmos as medias de valores das amostras diagnosticadas como benigmas ou malignas, para termos ideia de alguns limiares usados para analise dos dados

In [4]:
SELECT AVG(Raio) AS raio_media, AVG(Textura) AS textura_media, AVG(Perimetro) AS perimetro_media, AVG(Area) AS area_media,
       AVG(Suavidade) AS suavidade_media, AVG(Compacidade) AS compacidade_media, AVG(Concavidade) AS concavidade_media,
       AVG(PontosConcavos) AS pontosconcavos_media, AVG(Simetria) AS simetria_media, 
       AVG(DimensaoFractal) AS dimensaofractal_media
FROM Paciente
WHERE Diagnostico = 'M'

In [5]:
SELECT AVG(Raio) AS raio_media, AVG(Textura) AS textura_media, AVG(Perimetro) AS perimetro_media, AVG(Area) AS area_media,
       AVG(Suavidade) AS suavidade_media, AVG(Compacidade) AS compacidade_media, AVG(Concavidade) AS concavidade_media,
       AVG(PontosConcavos) AS pontosconcavos_media, AVG(Simetria) AS simetria_media, 
       AVG(DimensaoFractal) AS dimensaofractal_media
FROM Paciente
WHERE Diagnostico = 'B'

Outra consulta simples em que recuperamos os valores maximos e minimos para compararmos com as medias obtidas acima, onde talvez possamos obter alguns dados úteis

In [6]:
SELECT MAX(Raio) AS raio_max, MAX(Textura) AS textura_max, MAX(Perimetro) AS perimetro_max, MAX(Area) AS area_max,
       MAX(Suavidade) AS suavidade_max, MAX(Compacidade) AS compacidade_max, MAX(Concavidade) AS concavidade_max,
       MAX(PontosConcavos) AS pontosconcavos_max, MAX(Simetria) AS simetria_max, 
       MAX(DimensaoFractal) AS dimensaofractal_max
FROM Paciente
WHERE Diagnostico = 'M'

In [7]:
SELECT MAX(Raio) AS raio_max, MAX(Textura) AS textura_max, MAX(Perimetro) AS perimetro_max, MAX(Area) AS area_max,
       MAX(Suavidade) AS suavidade_max, MAX(Compacidade) AS compacidade_max, MAX(Concavidade) AS concavidade_max,
       MAX(PontosConcavos) AS pontosconcavos_max, MAX(Simetria) AS simetria_max, 
       MAX(DimensaoFractal) AS dimensaofractal_max
FROM Paciente
WHERE Diagnostico = 'B'

Algumas consultas para sabermos um pouco mais sobre o perfil das amostras

In [8]:
SELECT COUNT(*)
FROM Paciente

569

In [9]:
SELECT COUNT(*)
FROM Paciente
WHERE Diagnostico = 'M'

212

In [10]:
SELECT COUNT(*)
FROM Paciente
WHERE Diagnostico = 'B'

357

Consultas para tentarmos descobrir alguns padrões, tendo as medias como limiares

Consulta que checa quantas amostras benigmas possuem area maior que a média de amostras malignas, como podemos ver,
apenas uma amostra apresenta area maior, podemos concluir então que a area é um fator importante para indicar se é benigmo
ou maligno.

In [11]:
SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.Area > (SELECT AVG(P.Area) FROM Paciente P WHERE P.Diagnostico = 'M')

1

Mesma consulta para identificar o mesmo fato, só que agora para o perimetro.

In [12]:
SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.Perimetro > (SELECT AVG(P.Perimetro) FROM Paciente P WHERE P.Diagnostico = 'M')

0

Novamente para a concavidade.

In [13]:
SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.Concavidade > (SELECT AVG(P.Concavidade) FROM Paciente P WHERE P.Diagnostico = 'M')

5

Novamente para a compacidade

In [14]:
SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.Compacidade > (SELECT AVG(P.Compacidade) FROM Paciente P WHERE P.Diagnostico = 'M')

16

E novamente para os pontos concavos.

In [15]:
SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.PontosConcavos > (SELECT AVG(P.PontosConcavos) FROM Paciente P WHERE P.Diagnostico = 'M')

0

Consulta para verificar considerando os 5 fatores acima, de forma a checar quantas amostras benigmas fogem dos padrões identificados acima. E como podemos checar, nenhuma amostra benigma possui, numeros maiores que estes limiares definidos, sendo assim um possível método potente para identificaç

In [16]:
SELECT COUNT(*)
FROM Paciente
WHERE Diagnostico = 'B' AND Area > 978.37 AND Perimetro > 115.36 AND Concavidade > 0.16 AND Compacidade >  0.145 AND 
      PontosConcavos > 0.088

0

A mesma consulta só que levando em conta os erros.

In [18]:
SELECT COUNT(*)
FROM Paciente P, Erro E
WHERE P.Id = E.IdPaciente AND
      P.Diagnostico = 'B' AND 
      P.Area + E.Area_erro > 978.37 AND 
      P.Perimetro + E.Perimetro_erro > 115.36 AND
      P.Concavidade + E.Concavidade_erro > 0.16 AND
      P.Compacidade + E.Compacidade_erro >  0.145 AND 
      P.PontosConcavos + E.PontosConcavos_erro > 0.088

0

In [ ]:
Analise emcima da correlação entre todos atributos (iremos fazer da Area com todos outros pois ja temos uma boa noção do panorama geral apenas com essa analise e a realizada acima)
Lembrando que valores pertos de 1 ou -1 indicam forte correlação.
Link de referencia explicando o que é correlação entre variaveis.
http://www.futebolmetria.com/correla--o-estat-stica.html

In [ ]:
Ajeitamos de maneira que as correlações estao em ordem decrescente.

In [ ]:
Raio X Area

In [42]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Raio) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Raio*Raio) AS sumPer2,
               SUM(Area*Raio) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

0.9873571700566123

In [ ]:
Perimetro X Area

In [32]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Perimetro) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Perimetro*Perimetro) AS sumPer2,
               SUM(Area*Perimetro) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

0.9865068039913901

In [ ]:
Concavidade X Area

In [37]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Concavidade) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Concavidade*Concavidade) AS sumPer2,
               SUM(Area*Concavidade) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

0.6859828285519111

In [ ]:
PontosConcavos X Area

In [38]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(PontosConcavos) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(PontosConcavos*PontosConcavos) AS sumPer2,
               SUM(Area*PontosConcavos) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

0.8232688686524332

In [ ]:
Compacidade X Area

In [34]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Compacidade) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Compacidade*Compacidade) AS sumPer2,
               SUM(Area*Compacidade) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

0.4985016821524117

In [ ]:
Textura X Area

In [40]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Textura) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Textura*Textura) AS sumPer2,
               SUM(Area*Textura) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

0.321085696080207

In [ ]:
DimensaoFractal X Area

In [43]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(DimensaoFractal) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(DimensaoFractal*DimensaoFractal) AS sumPer2,
               SUM(Area*DimensaoFractal) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

-0.2831098116914262

In [ ]:
Suavidade X Area

In [39]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Suavidade) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Suavidade*Suavidade) AS sumPer2,
               SUM(Area*Suavidade) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

0.17702837725400117

In [ ]:
Simetria X Area

In [41]:
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Simetria) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Simetria*Simetria) AS sumPer2,
               SUM(Area*Simetria) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

0.1512930790351122